In [1]:
from work.unet.clarifruit_unet import unet_model_functions
from work.preprocess import data_functions

from keras.callbacks import ReduceLROnPlateau,EarlyStopping


Using TensorFlow backend.


In [2]:
params_dict = dict(

    train_path=r'D:\Clarifruit\cherry_stem\data\raw_data\with_maskes',
    test_path=r'D:\Clarifruit\cherry_stem\data\raw_data\images_orig',
    x_folder_name='image',
    y_folder_name='label',
    dest_path=r'D:\Clarifruit\cherry_stem\data\unet_data\training',
    weights_file_name='unet_cherry_stem.hdf5',

    data_gen_args=dict(rescale=1. / 255,
                       rotation_range=180,
                       brightness_range=(1., 1.),
                       width_shift_range=0.25,
                       height_shift_range=0.25,
                       shear_range=0.2,
                       zoom_range=0.2,
                       horizontal_flip=True,
                       vertical_flip=True,
                       fill_mode='nearest'),

    optimizer='Adam',
    optimizer_params=dict(lr=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy'],
    pretrained_weights=None,

    target_size=(256, 256),
    color_mode='grayscale',
    mask_color_mode='grayscale',
    batch_size=10,
    epochs=10,
    steps_per_epoch=3000,
    valdiation_split=0.2,
    validation_steps=300)




In [3]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2, min_lr=0.000001,
                              cooldown=1, verbose=1)

callbacks = [reduce_lr]
params_dict['callbacks'] = callbacks

model = unet_model_functions.ClarifruitUnet(**params_dict)


Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.


In [4]:
model.train_model(params_dict,True)


Epoch 1/10
3000/3000 [==============================] - 2869s 956ms/step - loss: 0.0387 - acc: 0.9629 - val_loss: 0.0617 - val_acc: 0.9548

Epoch 00001: loss improved from inf to 0.03878, saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-09-22_00-55-09\unet_cherry_stem.hdf5
Epoch 2/10
3000/3000 [==============================] - 2858s 953ms/step - loss: 0.0304 - acc: 0.9627 - val_loss: 0.0558 - val_acc: 0.9552

Epoch 00002: loss improved from 0.03878 to 0.03039, saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-09-22_00-55-09\unet_cherry_stem.hdf5
Epoch 3/10
3000/3000 [==============================] - 2853s 951ms/step - loss: 0.0293 - acc: 0.9626 - val_loss: 0.0590 - val_acc: 0.9526

Epoch 00003: loss improved from 0.03039 to 0.02922, saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-09-22_00-55-09\unet_cherry_stem.hdf5
Epoch 4/10
3000/3000 [==============================] - 2848s 949ms/step - loss: 0.0265 - acc: 0.9626 - 

KeyboardInterrupt: 

In [6]:
model.prediction()

In [2]:
src_path = r'D:\Clarifruit\cherry_stem\data\unet_data\training\2019-09-22_00-55-09'
params_dict = unet_model_functions.ClarifruitUnet.load_model(src_path)

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2, min_lr=0.000001,
                              cooldown=1, verbose=1)

early_stoping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

train_params = dict(

    data_gen_args=dict(rescale=1. / 255,
                       rotation_range=180,
                       brightness_range=[0.2, 1.],
                       width_shift_range=0.25,
                       height_shift_range=0.25,
                       shear_range=0.2,
                       zoom_range=[0.5,1.0],
                       horizontal_flip=True,
                       vertical_flip=True,
                       fill_mode='nearest'),

    callbacks = [reduce_lr,early_stoping],

    batch_size=10,
    epochs=10,
    steps_per_epoch=3000,
    valdiation_split=0.2,
    validation_steps=3000)

params_dict.update(train_params)

model = unet_model_functions.ClarifruitUnet(**params_dict)

Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.


In [3]:
model.train_model(params_dict,saveflag=True)

Epoch 1/10
3000/3000 [==============================] - 3707s 1s/step - loss: 0.0231 - acc: 0.9630 - val_loss: 0.0440 - val_acc: 0.9586

Epoch 00001: val_loss improved from inf to 0.04400, saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-09-22_23-55-20\unet_cherry_stem.hdf5
Epoch 2/10
3000/3000 [==============================] - 3697s 1s/step - loss: 0.0225 - acc: 0.9629 - val_loss: 0.0499 - val_acc: 0.9586

Epoch 00002: val_loss did not improve from 0.04400
Epoch 3/10
3000/3000 [==============================] - 3702s 1s/step - loss: 0.0222 - acc: 0.9630 - val_loss: 0.0515 - val_acc: 0.9584

Epoch 00003: val_loss did not improve from 0.04400
Epoch 4/10
3000/3000 [==============================] - 3706s 1s/step - loss: 0.0220 - acc: 0.9629 - val_loss: 0.0496 - val_acc: 0.9584

Epoch 00004: val_loss did not improve from 0.04400
Epoch 00004: early stopping


In [6]:
model.model.save_weights(r"D:\Clarifruit\cherry_stem\data\unet_data\training\2019-09-22_23-55-20\model.hdf5")

In [ ]:
model.prediction(threshold=0.5)